<a href="https://colab.research.google.com/github/weepingwillow2001/data_analysis_practice/blob/main/6%EC%9E%A5/6_1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

tables = pd.read_html("https://www.fdic.gov/bank/individual/failed/banklist.html")
# pd."read_html"()은 HTML 문서에서 <table> 태그로 정의된 모든 "표"를 찾아서 각각을 pandas DataFrame으로 변환해주는 함수입니다.

print(len(tables))  # len(tables)는 HTML 파일에서 찾은 "표"(table)의 "개수"를 찾음
# 1

failures = tables[0]

# 만약 HTML 문서에 여러 개의 표가 있다면:

# tables[0]: 첫 번째 표
# tables[1]: 두 번째 표
# tables[2]: 세 번째 표



# 설명: HTML 파싱을 위한 라이브러리들을 설치하고, pandas의 read_html 함수를 사용해 HTML 파일에서 표 데이터를 추출합니다.
# 이 파일에는 하나의 표가 있으며, 그 데이터를 failures 변수에 저장합니다.

1


In [ ]:
# 설명: 추출한 데이터의 앞부분을 확인하고, "Closing Date" 열을 datetime 형식으로 변환한 후
# 연도별로 파산한 은행의 수를 계산합니다.

failures.head()

close_timestamps = pd.to_datetime(failures["Closing Date"])  # "Closing Date" 열을 datetime 형식으로 변환
close_timestamps.dt.year.value_counts()

# dt 접근자는 다른 날짜 구성 요소에도 사용할 수 있습니다:

# dt.month: 월 추출
# dt.day: 일 추출
# dt.weekday: 요일 추출
# dt.quarter: 분기 추출

,count
Closing Date,
2023,5
2024,2
2020,2
2025,1


In [4]:
# 설명: lxml의 objectify 모듈을 사용해 XML 파일을 파싱합니다. 파일을 열고 파싱한 후, XML의 루트 요소를 가져옵니다.

from lxml import objectify

!git clone https://github.com/wesm/pydata-book.git
# 클론 후 파일 사용
path = "pydata-book/datasets/mta_perf/Performance_MNR.xml"

with open(path) as f:
    parsed = objectify.parse(f)

root = parsed.getroot()

Cloning into 'pydata-book'...
remote: Enumerating objects: 553, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 553 (delta 23), reused 3 (delta 3), pack-reused 525 (from 1)
Receiving objects: 100% (553/553), 63.76 MiB | 17.69 MiB/s, done.
Resolving deltas: 100% (209/209), done.
Updating files: 100% (208/208), done.


In [6]:
# 설명: XML 데이터를 파이썬 딕셔너리 리스트로 변환합니다.
# 각 INDICATOR 요소에 대해 특정 필드(skip_fields)를 제외한 모든 하위 요소의 값을 추출하고,
# 이를 pandas DataFrame으로 변환합니다.

data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
              "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


In [7]:
# 설명: pandas의 read_xml 함수를 사용해 동일한 XML 파일을 더 간단하게 DataFrame으로 변환합니다.
# 이 방법은 앞서 lxml.objectify를 사용한 방법보다 코드가 훨씬 간결합니다.

# 모든 데이터 가져오기
perf2 = pd.read_xml(path)

# 필요 없는 필드 제거하기
perf2 = perf2.drop(["PARENT_SEQ", "INDICATOR_SEQ", "DESIRED_CHANGE", "DECIMAL_PLACES"], axis=1, errors='ignore')  # DataFrame에 존재하지 않는 열을 drop하려고 할 때 발생할 수 있는 오류를 방지하기 위해 사용

# 결과 확인
perf2.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.00,96.90,95.00,96.90
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.00,96.00,95.00,95.00
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.00,96.30,95.00,96.90
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.00,96.80,95.00,98.30
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.00,96.60,95.00,95.80


In [ ]:
# pandas에서 함수들은 크게 두 그룹으로 나눌 수 있습니다:

# <선택/제거 함수> (drop, loc, iloc 등):

# axis=0 또는 axis='index': 행(rows)에 적용
# axis=1 또는 axis='columns': 열(columns)에 적용


# <연산/변환 함수> (apply, rank, mean, sum 등):

# axis=0 또는 axis='index': 열 방향으로 적용 (각 열에 대해 계산)
# axis=1 또는 axis='columns': 행 방향으로 적용 (각 행에 대해 계산)